**submit.py**

In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
from scipy.linalg import khatri_rao

################################
# Non Editable Region Starting #
################################
def my_fit(X_train, y0_train, y1_train):
    ################################
    #  Non Editable Region Ending  #
    ################################

    # Map challenges to higher dimensional space
    X_mapped = my_map(X_train)

    # Train model for Response0
    clf0 = LinearSVC()
    clf0.fit(X_mapped, y0_train)

    # Train model for Response1
    clf1 = LinearSVC()
    clf1.fit(X_mapped, y1_train)

    # Extract weights and biases
    w0 = clf0.coef_.flatten()
    b0 = clf0.intercept_[0]

    w1 = clf1.coef_.flatten()
    b1 = clf1.intercept_[0]

    return w0, b0, w1, b1

################################
# Non Editable Region Starting #
################################
def my_map(X):
    ################################
    #  Non Editable Region Ending  #
    ################################
    num_features = X.shape[1]
    num_samples = X.shape[0]
    num_mapped_features = num_features ** 2

    # Initialize the mapped features matrix
    mapped_features = np.zeros((num_samples, num_mapped_features))

    # Iterate over rows and compute Khatri-Rao product for each row
    for i in range(num_samples):
        row = X[i, :].reshape(1, -1)  # Reshape row to a 2D array
        row_result = khatri_rao(row.T, row.T)  # Compute Khatri-Rao for the row, transposed
        mapped_features[i, :] = row_result.flatten()

    return mapped_features

def validate():
    # Example file paths
    train_file = "public_trn.txt"
    test_file = "public_tst.txt"

    # Load data
    X_train, y0_train, y1_train = load_data(train_file)

    # Fit the model
    w0, b0, w1, b1 = my_fit(X_train, y0_train, y1_train)

    # Print the results
    print("Weights and bias for Response0:")
    print(w0, b0)
    print("Weights and bias for Response1:")
    print(w1, b1)

    # Load test data (assuming we want to validate on the same data format)
    X_test, y0_test, y1_test = load_data(test_file)

    # Map the test data
    X_test_mapped = my_map(X_test)

    # Predict responses
    y0_pred = np.sign(X_test_mapped @ w0 + b0)
    y1_pred = np.sign(X_test_mapped @ w1 + b1)

    # Calculate accuracy
    accuracy0 = np.mean((y0_pred > 0) == y0_test)
    accuracy1 = np.mean((y1_pred > 0) == y1_test)

    print("Test accuracy for Response0:", accuracy0)
    print("Test accuracy for Response1:", accuracy1)

def load_data(file_path):
    # Load training data
    data = np.loadtxt(file_path, delimiter=" ")
    X = data[:, :32]
    y0 = data[:, 32]
    y1 = data[:, 33]
    return X, y0, y1

if __name__ == "__main__":
    validate()

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Weights and bias for Response0:
[  1.05881585   0.0414974   -0.21852695 ...  -0.23522568   4.16288809
 -11.57132579] -0.44031316063134335
Weights and bias for Response1:
[ 0.32937379  0.56387337 -0.18190241 ...  0.24336342  0.82469004
 -1.77593718] -4.6175742173938135
Test accuracy for Response0: 0.9863
Test accuracy for Response1: 0.9943


**Question 6**

In [ ]:
import numpy as np
import time
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load data function
def load_data(file_path):
    data = np.loadtxt(file_path, delimiter=" ")
    X = data[:, :32]
    y0 = data[:, 32]
    y1 = data[:, 33]
    return X, y0, y1




    ################################
def my_map(X):
    ################################
    #  Non Editable Region Ending  #
    ################################
    num_features = X.shape[1]
    num_samples = X.shape[0]
    num_mapped_features = num_features ** 2

    # Initialize the mapped features matrix
    mapped_features = np.zeros((num_samples, num_mapped_features))

    # Iterate over rows and compute Khatri-Rao product for each row
    for i in range(num_samples):
        row = X[i, :].reshape(1, -1)  # Reshape row to a 2D array
        row_result = khatri_rao(row.T, row.T)  # Compute Khatri-Rao for the row, transposed
        mapped_features[i, :] = row_result.flatten()

    return mapped_features

# Training and evaluation function
def train_and_evaluate(X_train, y0_train, y1_train, X_test, y0_test, y1_test, model, **kwargs):
    X_train_mapped = my_map(X_train)
    X_test_mapped = my_map(X_test)

    clf0 = model(**kwargs)
    clf1 = model(**kwargs)

    start_time = time.time()
    clf0.fit(X_train_mapped, y0_train)
    clf1.fit(X_train_mapped, y1_train)
    training_time = time.time() - start_time

    y0_pred = clf0.predict(X_test_mapped)
    y1_pred = clf1.predict(X_test_mapped)

    accuracy0 = accuracy_score(y0_test, y0_pred)
    accuracy1 = accuracy_score(y1_test, y1_pred)

    return training_time, accuracy0, accuracy1

# Load data
X_train, y0_train, y1_train = load_data("public_trn.txt")
X_test, y0_test, y1_test = load_data("public_tst.txt")

# Experiments setup
experiments = [
    {"model": LinearSVC, "name": "LinearSVC_default", "kwargs": {}},
    {"model": LinearSVC, "name": "LinearSVC_hinge", "kwargs": {"loss": "hinge"}},
    {"model": LinearSVC, "name": "LinearSVC_squared_hinge", "kwargs": {"loss": "squared_hinge"}},
    {"model": LinearSVC, "name": "LinearSVC_C_high", "kwargs": {"C": 10}},
    {"model": LinearSVC, "name": "LinearSVC_C_low", "kwargs": {"C": 0.1}},
    {"model": LinearSVC, "name": "LinearSVC_tol_high", "kwargs": {"tol": 1e-1}},
    {"model": LinearSVC, "name": "LinearSVC_tol_low", "kwargs": {"tol": 1e-5}},
    {"model": LinearSVC, "name": "LinearSVC_penalty_l2", "kwargs": {"penalty": "l2"}},
    {"model": LinearSVC, "name": "LinearSVC_penalty_l1", "kwargs": {"penalty": "l1", "dual": False}},
    {"model": LogisticRegression, "name": "LogisticRegression_default", "kwargs": {}},
    {"model": LogisticRegression, "name": "LogisticRegression_C_high", "kwargs": {"C": 10}},
    {"model": LogisticRegression, "name": "LogisticRegression_C_low", "kwargs": {"C": 0.1}},
    {"model": LogisticRegression, "name": "LogisticRegression_tol_high", "kwargs": {"tol": 1e-1}},
    {"model": LogisticRegression, "name": "LogisticRegression_tol_low", "kwargs": {"tol": 1e-5}},
    {"model": LogisticRegression, "name": "LogisticRegression_penalty_l2", "kwargs": {"penalty": "l2"}},
    {"model": LogisticRegression, "name": "LogisticRegression_penalty_l1", "kwargs": {"penalty": "l1", "solver": "liblinear"}}
]

results = []

for experiment in experiments:
    training_time, accuracy0, accuracy1 = train_and_evaluate(
        X_train, y0_train, y1_train, X_test, y0_test, y1_test, experiment["model"], **experiment["kwargs"]
    )
    results.append({
        "Experiment": experiment["name"],
        "Training Time (s)": training_time,
        "Accuracy0": accuracy0,
        "Accuracy1": accuracy1
    })

import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)


NameError: name 'khatri_rao' is not defined

**Question 6: Code 2**

In [ ]:
  import numpy as np
  import time
  from sklearn.svm import LinearSVC
  from sklearn.linear_model import LogisticRegression
  from sklearn.metrics import accuracy_score
  from sklearn.preprocessing import StandardScaler
  import pandas as pd
  from sklearn.exceptions import ConvergenceWarning
  import warnings
  from scipy.linalg import khatri_rao

  # Load data function
  def load_data(file_path):
      data = np.loadtxt(file_path, delimiter=" ")
      X = data[:, :32]
      y0 = data[:, 32]
      y1 = data[:, 33]
      return X, y0, y1

  # Map challenges to higher dimensional space using Khatri-Rao product

def my_map(X):

    num_features = X.shape[1]
    num_samples = X.shape[0]
    num_mapped_features = num_features ** 2


    mapped_features = np.zeros((num_samples, num_mapped_features))

    for i in range(num_samples):
        row = X[i, :].reshape(1, -1)
        row_result = khatri_rao(row.T, row.T)
        mapped_features[i, :] = row_result.flatten()

    return mapped_features

  # Training and evaluation function
  def train_and_evaluate(X_train, y0_train, y1_train, X_test, y0_test, y1_test, model, **kwargs):
      scaler = StandardScaler()
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)

      X_train_mapped = my_map(X_train)
      X_test_mapped = my_map(X_test)

      clf0 = model(max_iter=5000, **kwargs)
      clf1 = model(max_iter=5000, **kwargs)

      start_time = time.time()
      with warnings.catch_warnings():
          warnings.simplefilter("ignore", category=ConvergenceWarning)
          clf0.fit(X_train_mapped, y0_train)
          clf1.fit(X_train_mapped, y1_train)
      training_time = time.time() - start_time

      y0_pred = clf0.predict(X_test_mapped)
      y1_pred = clf1.predict(X_test_mapped)

      accuracy0 = accuracy_score(y0_test, y0_pred)
      accuracy1 = accuracy_score(y1_test, y1_pred)

      return training_time, accuracy0, accuracy1

  # Load data
  X_train, y0_train, y1_train = load_data("public_trn.txt")
  X_test, y0_test, y1_test = load_data("public_tst.txt")

  # Experiments setup
  experiments = [
      {"model": LinearSVC, "name": "LinearSVC_default", "kwargs": {}},
      {"model": LinearSVC, "name": "LinearSVC_hinge", "kwargs": {"loss": "hinge"}},
      {"model": LinearSVC, "name": "LinearSVC_squared_hinge", "kwargs": {"loss": "squared_hinge"}},
      {"model": LinearSVC, "name": "LinearSVC_C_high", "kwargs": {"C": 10}},
      {"model": LinearSVC, "name": "LinearSVC_C_low", "kwargs": {"C": 0.1}},
      {"model": LinearSVC, "name": "LinearSVC_tol_high", "kwargs": {"tol": 1e-1}},
      {"model": LinearSVC, "name": "LinearSVC_tol_low", "kwargs": {"tol": 1e-5}},
      {"model": LinearSVC, "name": "LinearSVC_penalty_l2", "kwargs": {"penalty": "l2"}},
      {"model": LinearSVC, "name": "LinearSVC_penalty_l1", "kwargs": {"penalty": "l1", "dual": False}},
      {"model": LogisticRegression, "name": "LogisticRegression_default", "kwargs": {}},
      {"model": LogisticRegression, "name": "LogisticRegression_C_high", "kwargs": {"C": 10}},
      {"model": LogisticRegression, "name": "LogisticRegression_C_low", "kwargs": {"C": 0.1}},
      {"model": LogisticRegression, "name": "LogisticRegression_tol_high", "kwargs": {"tol": 1e-1}},
      {"model": LogisticRegression, "name": "LogisticRegression_tol_low", "kwargs": {"tol": 1e-5}},
      {"model": LogisticRegression, "name": "LogisticRegression_penalty_l2", "kwargs": {"penalty": "l2"}},
      {"model": LogisticRegression, "name": "LogisticRegression_penalty_l1", "kwargs": {"penalty": "l1", "solver": "liblinear"}}
  ]

  results = []

  for experiment in experiments:
      training_time, accuracy0, accuracy1 = train_and_evaluate(
          X_train, y0_train, y1_train, X_test, y0_test, y1_test, experiment["model"], **experiment["kwargs"]
      )
      results.append({
          "Experiment": experiment["name"],
          "Training Time (s)": training_time,
          "Accuracy0": accuracy0,
          "Accuracy1": accuracy1
      })

  results_df = pd.DataFrame(results)
  print(results_df)

Another approach


In [ ]:
import numpy as np
from sklearn.svm import LinearSVC
from scipy.linalg import khatri_rao
import time

################################
# Non Editable Region Starting #
################################
def my_fit(X_train, y0_train, y1_train):
    ################################
    #  Non Editable Region Ending  #
    ################################

    # Map challenges to higher dimensional space
    X_mapped = my_map(X_train)

    # Train model for Response0
    clf0 = LinearSVC(loss='squared_hinge')
    clf0.fit(X_mapped, y0_train)

    # Train model for Response1
    clf1 = LinearSVC(loss='squared_hinge')
    clf1.fit(X_mapped, y1_train)

    # Extract weights and biases
    w0 = clf0.coef_.flatten()
    b0 = clf0.intercept_[0]

    w1 = clf1.coef_.flatten()
    b1 = clf1.intercept_[0]

    return w0, b0, w1, b1

################################
# Non Editable Region Starting #
################################
def my_map(X):
    ################################
    #  Non Editable Region Ending  #
    ################################
    num_features = X.shape[1]
    num_samples = X.shape[0]
    num_mapped_features = num_features ** 2

    # Initialize the mapped features matrix
    mapped_features = np.zeros((num_samples, num_mapped_features))

    # Iterate over rows and compute Khatri-Rao product for each row
    for i in range(num_samples):
        row = X[i, :].reshape(1, -1)  # Reshape row to a 2D array
        row_result = khatri_rao(row.T, row.T)  # Compute Khatri-Rao for the row, transposed
        mapped_features[i, :] = row_result.flatten()

    return mapped_features

def validate():
    # Example file paths
    train_file = "public_trn.txt"
    test_file = "public_tst.txt"

    # Load data
    X_train, y0_train, y1_train = load_data(train_file)
    start_time = time.time()
    # Fit the model
    w0, b0, w1, b1 = my_fit(X_train, y0_train, y1_train)
    training_time = time.time() - start_time
    # Print the results
    print("Weights and bias for Response0:")
    print(w0, b0)
    print("Weights and bias for Response1:")
    print(w1, b1)

    # Load test data (assuming we want to validate on the same data format)
    X_test, y0_test, y1_test = load_data(test_file)

    # Map the test data
    X_test_mapped = my_map(X_test)

    # Predict responses
    y0_pred = np.sign(X_test_mapped @ w0 + b0)
    y1_pred = np.sign(X_test_mapped @ w1 + b1)

    # Calculate accuracy
    accuracy0 = np.mean((y0_pred > 0) == y0_test)
    accuracy1 = np.mean((y1_pred > 0) == y1_test)

    print("Test accuracy for Response0:", accuracy0)
    print("Test accuracy for Response1:", accuracy1)
    print("Training time:", training_time)

def load_data(file_path):
    # Load training data
    data = np.loadtxt(file_path, delimiter=" ")
    X = data[:, :32]
    y0 = data[:, 32]
    y1 = data[:, 33]
    return X, y0, y1

if __name__ == "__main__":
    validate()

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Weights and bias for Response0:
[  1.04937987   0.0425067   -0.2190335  ...  -0.23247338   4.16335398
 -11.57396672] -0.4344516324798803
Weights and bias for Response1:
[ 0.33243872  0.56453659 -0.179414   ...  0.24640007  0.82631344
 -1.77302364] -4.617000923881357
Test accuracy for Response0: 0.9847
Test accuracy for Response1: 0.9943
Training time: 7.6211066246032715


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
